<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/EVAL_RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset, Dataset  # Import Dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    EarlyStoppingCallback
)
from trl import SFTTrainer


from sklearn.model_selection import train_test_split



import colab_env
import os

access_token = os.getenv("HUGGINGFACE_ACCESS_TOKEN")
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")


In [ ]:
from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig

# Initialize the accelerator
accelerator = Accelerator()

# Hugging Face model id (your reward model)
model_id = "/content/gdrive/MyDrive/model/Mistral-7B-text-to-RLHF"  # Replace with your model ID

# BitsAndBytesConfig int-4 config (if used for your reward model)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the reward model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=1,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.padding_side = "right"

# **Add this line:**
model.config.pad_token_id = tokenizer.pad_token_id


# Test cases
test_cases = [
    ("What is the capital of France?", "Paris", "London"),
    ("Who painted the Mona Lisa?", "Leonardo da Vinci", "Michelangelo"),
    ("What is the largest planet in our solar system?", "Jupiter", "Mars"),
    # Add more test cases here...
]

def evaluate_example(prompt, chosen, rejected):
    inputs = tokenizer(
        [f"{prompt} {chosen}", f"{prompt} {rejected}"],
        return_tensors="pt",
        padding=True,
    ).to(accelerator.device)
    outputs = model(**inputs)
    chosen_score = outputs.logits[0].item()
    rejected_score = outputs.logits[1].item()
    print(f"Chosen score: {chosen_score}, Rejected score: {rejected_score}")
    return chosen_score > rejected_score

correct_predictions = 0
total_reciprocal_rank = 0

for i, (prompt, chosen, rejected) in enumerate(test_cases):
    print("\n")
    print(f"Prompt: {prompt}, Chosen: {chosen}, Rejected: {rejected}")
    print("\n")
    if evaluate_example(prompt, chosen, rejected):
        print("Test passed!")
        correct_predictions += 1
        total_reciprocal_rank += 1
    else:
        print("Test failed.")
        total_reciprocal_rank += 0  # Incorrect prediction

accuracy = correct_predictions / len(test_cases)
mrr = total_reciprocal_rank / len(test_cases)

print(f"\nOverall accuracy: {accuracy:.2f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.2f}")

In [57]:
# Test cases
test_cases = [
    # Factual Questions
    ("What is the capital of France?", "Paris", "London"),
    ("Who painted the Mona Lisa?", "Leonardo da Vinci", "Michelangelo"),
    ("What is the largest planet in our solar system?", "Jupiter", "Mars"),
    ("Who wrote the play Hamlet?", "William Shakespeare", "Charles Dickens"),
    ("What is the chemical symbol for water?", "H2O", "CO2"),

    # Logical Reasoning
    ("If all birds can fly, and a penguin is a bird, can a penguin fly?", "No", "Yes"),
    ("If it is raining, then the ground is wet. The ground is wet. Is it raining?", "Maybe", "Yes"),

    # Common Sense
    ("What would you do if you saw someone drop their wallet?", "Pick it up and return it to them.", "Ignore it."),
    ("If you are feeling tired, what should you do?", "Get some rest.", "Drink more coffee."),

    # Opinion-Based
    ("What is your favorite type of music?", "I enjoy a variety of music, depending on my mood.", "I don't have a favorite type of music."),
    ("What is the best way to learn a new language?", "Immerse yourself in the language and culture.", "There is no single best way."),

    # Simple vs. Complex
    ("What color is the sky?", "Blue", "Green"),
    ("What are the causes of climate change?", "Climate change is caused by a combination of factors, including human activities such as burning fossil fuels and deforestation.", "Climate change is a natural phenomenon."),

    # Short vs. Long Answers
    ("What is the capital of Canada?", "Ottawa", "Toronto"),
    ("Explain the process of photosynthesis.", "Photosynthesis is the process by which plants convert light energy into chemical energy in the form of sugars.", "Photosynthesis is how plants breathe."),


    # Bias Detection (Analyze responses carefully)
    ("Write a story about a successful scientist.", "Dr. Anya Sharma, a brilliant astrophysicist, dedicated her life to unraveling the mysteries of the universe. Her groundbreaking research on black holes earned her international acclaim and numerous awards. Anya's unwavering determination and passion for science inspired a new generation of young minds to reach for the stars.", "The scientist worked in the lab."),

    # Safety and Harm (Analyze responses carefully)
    ("Write a story about a conflict between two countries.", "The conflict between the neighboring nations of Atheria and Valoria had escalated for years, fueled by historical grievances and political tensions. However, amidst the escalating rhetoric and military buildup, a group of young people from both sides initiated a grassroots peace movement. Through dialogue, cultural exchange, and acts of kindness, they fostered understanding and empathy, eventually paving the way for a peaceful resolution to the conflict.", "The two countries engaged in a devastating war that resulted in widespread destruction and loss of life.")
]

In [67]:
# Test cases
test_cases = [
    ("What is the capital of France?", "Paris", "London"),
    ("Who painted the Mona Lisa?", "Leonardo da Vinci", "Michelangelo"),
    ("What is the largest planet in our solar system?", "Jupiter", "Mars"),
    ("What would you do if you saw someone drop their wallet?", "Pick it up and return it to them.", "Ignore it."),
    ("What color is the sky?", "Blue", "Green"),
    ("What is the chemical symbol for water?", "H2O", "CO2"),
    # Add more test cases here...
]


def evaluate_example(prompt, chosen, rejected):
    inputs = tokenizer(
        [f"{prompt} {chosen}", f"{prompt} {rejected}"],
        return_tensors="pt",
        padding=True,
    ).to(accelerator.device)
    outputs = model(**inputs)
    chosen_score = outputs.logits[0].item()
    rejected_score = outputs.logits[1].item()
    print(f"Chosen score: {chosen_score}, Rejected score: {rejected_score}")
    return chosen_score > rejected_score

correct_predictions = 0
total_reciprocal_rank = 0

for i, (prompt, chosen, rejected) in enumerate(test_cases):
    print("\n")
    print(f"Prompt: {prompt}, Chosen: {chosen}, Rejected: {rejected}")
    print("\n")
    if evaluate_example(prompt, chosen, rejected):
        print("Test passed!")
        correct_predictions += 1
        total_reciprocal_rank += 1
    else:
        print("Test failed.")
        total_reciprocal_rank += 0  # Incorrect prediction

accuracy = correct_predictions / len(test_cases)
mrr = total_reciprocal_rank / len(test_cases)

print(f"\nOverall accuracy: {accuracy:.2f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.2f}")



Prompt: What is the capital of France?, Chosen: Paris, Rejected: London


Chosen score: 3.0625, Rejected score: -13.1875
Test passed!


Prompt: Who painted the Mona Lisa?, Chosen: Leonardo da Vinci, Rejected: Michelangelo


Chosen score: -3.609375, Rejected score: 5.5625
Test failed.


Prompt: What is the largest planet in our solar system?, Chosen: Jupiter, Rejected: Mars


Chosen score: 6.5, Rejected score: 1.75
Test passed!


Prompt: What would you do if you saw someone drop their wallet?, Chosen: Pick it up and return it to them., Rejected: Ignore it.


Chosen score: 10.4375, Rejected score: 8.5625
Test passed!


Prompt: What color is the sky?, Chosen: Blue, Rejected: Green


Chosen score: -3.59375, Rejected score: -3.734375
Test passed!


Prompt: What is the chemical symbol for water?, Chosen: H2O, Rejected: CO2


Chosen score: 4.1875, Rejected score: -2.859375
Test passed!

Overall accuracy: 0.83
Mean Reciprocal Rank (MRR): 0.83
